In [4]:
system_name='646_SDS_on_AU_111_51x30x21_hemicylinders_with_counterion'
pdb_segment_chunk_glob_pattern = '*_[0-9][0-9][0-9].pdb'
a='tar -czf {:s} {:s}'.format(system_name + '_segments.tar.gz',pdb_segment_chunk_glob_pattern )

In [5]:
a

'tar -czf 646_SDS_on_AU_111_51x30x21_hemicylinders_with_counterion_segments.tar.gz *_[0-9][0-9][0-9].pdb'

In [49]:
from fireworks import Firework, FWorker, LaunchPad, ScriptTask, FileTransferTask
from fireworks.user_objects.firetasks.filepad_tasks import AddFilesTask, GetFilesTask
from fireworks.user_objects.firetasks.jlh_tasks import CmdTask
    
launchpad = LaunchPad(
            host='localhost',
            port=27017,
            name='fireworks-jhoermann',
            username='fireworks',
            password='japanese1')

In [50]:
task1 = FileTransferTask({'files': ['/home/bramos/lammps/Lab/05_Gold_cluster/Test05/TestF/cluster_3871.lammps','/home/bramos/lammps/Lab/05_Gold_cluster/Test05/TestF/Au-Grochola-JCP05.eam.alloy', '/home/bramos/lammps/Lab/05_Gold_cluster/Test05/TestF/A_minimize_cluster.in'], 'dest': '.', 'mode': 'copy', 'ignore_errors': False})
task2 = ScriptTask.from_str('source /etc/profile; module load LAMMPS; mpirun -n 4 lmp -in A_minimize_cluster.in',{'shell_exe': "/bin/bash"})
# create the Firework
fw1= Firework([task1, task2], spec={'_category': "bwcloud_noqueue",'_files_out': {'data_file': "A_minimized_cluster.lammps"}})
#launchpad.add_wf(firework)


In [57]:
lmp_cmd=CmdTask({'cmd': 'lmp',
                 'opt':
                 {'-in lmp_equilibration_nvt.input',
                  '-v baseName '+ baseName,
                  '-v dataFile '+ dataFile}
                })

extract_thermo_ft =  CmdTask({'cmd': 'extract_thermo.sh',
                              'opt':
                              {system_name + '_minimization.log',
                               system_name + '_minimization_thermo.out'},
                              'stderr_file':'extract_thermo.err',
                              'stdout_file':'extract_thermo.out',
                              'fizzle_bad_rc':'false',
                              'use_shell':'true'
                             }


fw1= Firework([a], spec={'_category': "bwcloud_noqueue",'_files_out': {'data_file': "A_minimized_cluster.lammps"}})
fw1.as_dict()

{'spec': {'_category': 'bwcloud_noqueue',
  '_files_out': {'data_file': 'A_minimized_cluster.lammps'},
  '_tasks': [{'cmd': 'lmp',
    'opt': "{'-v baseName baseName', '-in lmp_equilibration_nvt.input', '-v dataFile dataFile'}",
    '_fw_name': 'CmdTask'}]},
 'fw_id': -14,
 'created_on': '2019-06-15T19:02:46.945587',
 'updated_on': '2019-06-15T19:02:46.945589',
 'name': 'Unnamed FW'}

In [ ]:
    fw = Firework(
            [
                lmp_cmd
            ],
            spec={
                "_category": "nemo_queue_offline",
                "system_name":  system_name,
                "_files_in":  { "lmp_header": 'lmp_header.input', 
                                "lmp_production": 'lmp_production.input', 
                                'lmp_production_mixed': 'lmp_production_mixed.input',
                                "datafile": 'datafile.lammps'},
                "step"     :    "production",
                "total_steps":  total_steps,
            },
            name="{:s}_produtcion_{:d}".format(system_name, total_steps) ,
            parents=[get_input_files_fw]
    )

In [ ]:
def minimize(system_name,
        lmp_suffix_template='-v baseName {baseName:s} -v dataFile {dataFile:s}'):
        """
        Sample for lmp_suffix: for a call like
            srun lmp -in lmp_minimization.input \
                -v has_indenter 1 -v robust_minimization 0 -v pbc2d 1 \
                -v baseName 377_SDS_on_AU_111_51x30x2_monolayer \
                -v dataFile 377_SDS_on_AU_111_51x30x2_monolayer.lammps
        set lmp_suffix='-v has_indenter 1 -v robust_minimization 0 -v pbc2d 1 \
            -v baseName {baseName:s} -v dataFile {dataFile:s}'
        """
        
        consecutive_fw_list = []

        get_input_files_ft = GetFilesTask( {
                'identifiers':    [ 'lmp_header.input', 
                                   'lmp_minimization.input',
                                   'extract_thermo.sh',
                                   system_name+'_psfgen.data' ] ,
                'new_file_names': [ 'lmp_header.input',
                                   'lmp_minimization.input',
                                   'extract_thermo.sh',
                                   'datafile.lammps']} )
        
        
        get_input_files_fw = Firework( get_input_files_ft,
            spec = {
                "_category":  "nemo_noqueue",
                "system_name" : system_name,
                "_files_out":  {"lmp_header":'lmp_header.input', 
                                "lmp_minimization":'lmp_minimization.input', 
                                "extract_thermo":'extract_thermo.sh', 
                                "datafile": 'datafile.lammps'},
                "step"   :     "get_files",
                },
            name=(system_name + "get_files") 
        )
        consecutive_fw_list.append(get_input_files_fw)
        
        
        lmp_cmd= ' '.join((
            'module load lammps/08May19-git-master-gnu-7.3-openmpi-3.1-colvars-08May19;',
            'mpirun ${{MPIRUN_OPTIONS}} lmp -in '+ 'lmp_minimization.input', lmp_suffix_template.format(
                baseName= system_name, dataFile='datafile.lammps' ) ))

        minimization_ft =  ScriptTask.from_str(
            lmp_cmd,
            {
                'stdout_file':  'lmp_minimization.out',
                'stderr_file':  'lmp_minimization.err',
                'use_shell':    True,
                'fizzle_bad_rc':True
            })

        extract_thermo_ft =  ScriptTask.from_str(
            'bash extract_thermo.sh {:s} {:s}'.format(
                system_name + '_minimization.log',
                system_name + '_minimization_thermo.out'),
                {
                    'use_shell':    True, 'fizzle_bad_rc': False
                } )

        fw = Firework(
            [
                minimization_ft,
                extract_thermo_ft
            ],
            spec={
                "_category":    "nemo_queue_offline",
                "_files_in":  {"lmp_header":'lmp_header.input', 
                                "lmp_minimization":'lmp_minimization.input', 
                                "extract_thermo":'extract_thermo.sh', 
                                "datafile": 'datafile.lammps'},
                "_files_out":  {"lmp_minimized":system_name + '_minimized.lammps'},
                "system_name": system_name,
                "step"     : "minimization"
            },
            name="{:s}_minimzation".format(system_name),
            parents=[get_input_files_fw] )
        
        consecutive_fw_list.append(fw)
        
        add_output_files_ft= AddFilesTask({'compress':True ,
                                           'identifier': [ system_name + '_minimized.lammps'], 
                                           'paths': [ system_name + '_minimized.lammps']})
        
        
        add_output_files_fw = Firework(add_output_files_ft,
            spec = {
                "_category":  "nemo_noqueue",
                "system_name" : system_name,
                "_files_in":  { "lmp_minimized":system_name + '_minimized.lammps' },
                "step"   :     "add_output_files",
                },
            name=(system_name + "add_output_files"),
            parents=[fw]
       )
        consecutive_fw_list.append(add_output_files_fw)
        
        parent_links = { consecutive_fw_list[i] : consecutive_fw_list[i+1] \
                        for i in range(len(consecutive_fw_list)-1) }

        return Workflow( consecutive_fw_list, parent_links,
            name="{:s}_min_wf".format(system_name) )

    
def nvtEquilibrate(system_name,
        lmp_suffix_template='-v baseName {baseName:s} -v dataFile {dataFile:s}'):
        """
        Sample for lmp_suffix: for a call like
            srun lmp -in lmp_equilibration_nvt.input \
                -v has_indenter 0 -v pbc2d 0 -v reinitialize_velocities 1\
                -v baseName 377_SDS_on_AU_111_51x30x2_monolayer \
                -v dataFile 377_SDS_on_AU_111_51x30x2_monolayer.lammps
        set lmp_suffix='-v has_indenter 0 -v reinitialize_velocities 1 \
            -v baseName {baseName:s} -v dataFile {dataFile:s}'
        """

        consecutive_fw_list = []
        
        get_input_files_ft = GetFilesTask( {
                'identifiers':    ['lmp_header.input', 
                                   'lmp_equilibration_nvt.input', 
                                   'extract_thermo.sh', 
                                   system_name + '_minimized.lammps'
                                  ],
                'new_file_names': ['lmp_header.input', 
                                   'lmp_equilibration_nvt.input',
                                   'extract_thermo.sh',
                                   'datafile.lammps']} )
        
        get_input_files_fw = Firework( get_input_files_ft,
            spec = {
                "_category":  "nemo_noqueue",
                "system_name" : system_name,
                "_files_out":  {"lmp_header":'lmp_header.input', 
                                "lmp_equilibration":'lmp_equilibration.input', 
                                "extract_thermo":'extract_thermo.sh', 
                                "datafile": 'datafile.lammps'},
                "step"   :     "get_files",
                },
            name=(system_name + "get_files") 
        )
        consecutive_fw_list.append(get_input_files_fw)
  
        lmp_cmd = ' '.join((
            'module load lammps/08May19-git-master-gnu-7.3-openmpi-3.1-colvars-08May19;',
            'mpirun ${{MPIRUN_OPTIONS}} lmp -in '+ 'lmp_equilibration_nvt.input' , lmp_suffix_template.format(
                baseName= system_name, dataFile='datafile.lammps' ) ) )

        lmp_ft =  ScriptTask.from_str(
            lmp_cmd,
            {
                'stdout_file':  'lmp_nvtEquilibration.out',
                'stderr_file':  'lmp_nvtEquilibration.err',
                'use_shell':    True,
                'fizzle_bad_rc':True
            })
        
        lmp_cmd=CmdTask()
        
    - _fw_name: CmdTask
      fizzle_bad_rc: true
      cmd: lmp
      opt:
      - -in lmp_equilibration_nvt.input
      - -v baseName {baseName:s}
      - -v dataFile {dataFile:s}
      stderr_file: strip.err
      stdout_file: strip.out
      use_shell: true

        extract_thermo_ft =  ScriptTask.from_str(
            'bash extract_thermo.sh {:s} {:s}'.format(
                system_name + '_nvtEquilibration.log',
                system_name + '_nvtEquilibration_thermo.out'),
                {
                    'use_shell':    True, 'fizzle_bad_rc': False
                } )

        fw = Firework(
            [
                lmp_ft,
                extract_thermo_ft
            ],
            spec={
             
                "_category":    "nemo_queue_offline",
                "system_name": system_name,
                "_files_in":  { "lmp_header":'lmp_header.input', 
                                "lmp_equilibration":'lmp_equilibration_nvt.input', 
                                "extract_thermo":'extract_thermo.sh', 
                                "datafile": 'datafile.lammps'},
                "_files_out":  {"lmp_nvt": system_name + '_nvtEquilibrated.lammps'},
                "step"     :   "equilibration_nvt",
            },
            name="{:s}_equilibration_nvt".format(system_name),
            parents=[get_input_files_fw] )
        
        
        consecutive_fw_list.append(fw)
    
        add_output_files_ft= AddFilesTask({'compress':True ,
                                           'identifier': [ system_name + '_nvtEquilibrated.lammps'],
                                           'paths': [ system_name + '_nvtEquilibrated.lammps']})
        
        
        add_output_files_fw = Firework(add_output_files_ft,
            spec = {
                "_category":  "nemo_noqueue",
                "system_name" : system_name,
                "_files_in":  { "lmp_nvt": system_name + '_nvtEquilibrated.lammps'},
                "step"   :     "add_output_files",
                },
            name=(system_name + "add_output_files"),
            parents=[fw]
       )
        consecutive_fw_list.append(add_output_files_fw)
        
        parent_links = { consecutive_fw_list[i] : consecutive_fw_list[i+1] \
                        for i in range(len(consecutive_fw_list)-1) }

        return Workflow( consecutive_fw_list, parent_links,
            name="{:s}_nvt_wf".format(system_name) )
    


def nptEquilibrate(system_name,
        lmp_suffix_template='-v baseName {baseName:s} -v dataFile {dataFile:s}'):
        """
        Sample for lmp_suffix: for a call like
            srun lmp -in lmp_equilibration_npt.input \
                -v has_indenter 1 -v pbc2d 0 -v reinitialize_velocities 1\
                -v baseName 377_SDS_on_AU_111_51x30x2_monolayer \
                -v dataFile 377_SDS_on_AU_111_51x30x2_monolayer.lammps
        set lmp_suffix='-v has_indenter 1 -v reinitialize_velocities 1 \
            -v baseName {baseName:s} -v dataFile {dataFile:s}'
        """
        consecutive_fw_list = []

        get_input_files_ft = GetFilesTask( {
                'identifiers':    ['lmp_header.input', 
                                   'lmp_equilibration_npt.input', 
                                   'extract_thermo.sh',
                                   system_name + '_nvtEquilibrated.lammps'
                                  ],
                'new_file_names': [ 'lmp_header.input', 
                                   'lmp_equilibration_npt.input', 
                                   'extract_thermo.sh',
                                   'datafile.lammps']} )
        
        get_input_files_fw = Firework( get_input_files_ft,
            spec = {
                "_category":  "nemo_noqueue",
                "system_name" : system_name,
                "_files_out":  {"lmp_header":'lmp_header.input', 
                                "lmp_equilibration":'lmp_equilibration_npt.input', 
                                "extract_thermo":'extract_thermo.sh',
                                "datafile": 'datafile.lammps'},
                "step"   :     "get_files",
                },
            name=(system_name + "get_files")
                                      
        )
        consecutive_fw_list.append(get_input_files_fw)
        
        lmp_cmd = ' '.join((
            'module load lammps/08May19-git-master-gnu-7.3-openmpi-3.1-colvars-08May19;',
            'mpirun ${{MPIRUN_OPTIONS}} lmp -in '+ 'lmp_equilibration_npt.input' , lmp_suffix_template.format(
                baseName= system_name, dataFile='datafile.lammps' ) ) )

        lmp_ft =  ScriptTask.from_str(
            lmp_cmd,
            {
                'stdout_file':  'lmp_nptEquilibration.out',
                'stderr_file':  'lmp_nptEquilibration.err',
                'use_shell':    True,
                'fizzle_bad_rc':True
            })

        extract_thermo_ft =  ScriptTask.from_str(
            'bash extract_thermo.sh {:s} {:s}'.format(
                system_name + '_nptEquilibration.log',
                system_name + '_nptEquilibration_thermo.out'),
                {
                    'use_shell':    True, 'fizzle_bad_rc': False
                } )

        fw = Firework(
            [
                lmp_ft,
                extract_thermo_ft
            ],
            spec={
                "_category":   "nemo_queue_offline",
                "system_name": system_name,
                "step"     :   "equilibration_npt",
                "_files_in":  {"lmp_header":'lmp_header.input', 
                                "lmp_equilibration":'lmp_equilibration_npt.input', 
                                "extract_thermo":'extract_thermo.sh',
                                "datafile": 'datafile.lammps'},
                "_files_out": {"lmp_npt":  system_name + '_nptEquilibrated.lammps'}
            },
            name="{:s}_equilibration_npt".format(system_name),
            parents=[get_input_files_fw])
        
        consecutive_fw_list.append(fw)
        
        add_output_files_ft= AddFilesTask({'compress':True ,
                                           'identifier': [ system_name + '_nptEquilibrated.lammps'], 
                                           'paths': [ system_name + '_nptEquilibrated.lammps']})
        
        add_output_files_fw = Firework(add_output_files_ft,
            spec = {
                "_category":  "nemo_noqueue",
                "system_name" : system_name,
                "_files_in":  { "lmp_npt": system_name + '_nptEquilibrated.lammps'},
                "step"   :     "add_output_files",
                },
            name=(system_name + "add_output_files"),
            parents=[fw]
       )
        consecutive_fw_list.append(add_output_files_fw)
        
        parent_links = { consecutive_fw_list[i] : consecutive_fw_list[i+1] \
                        for i in range(len(consecutive_fw_list)-1) }

        return Workflow( consecutive_fw_list, parent_links,
            name="{:s}_npt_wf".format(system_name) )

def production(system_name,
        total_steps      = 5000000, # time steps, 5 mio ~ 10 ns
        lmp_suffix_template=' '.join((
            '-v baseName {baseName:s} -v dataFile {dataFile:s}',
            '-v has_indenter 0 -v pbc2d 0 -v mpiio 0 -v use_colvars 0')) ):
        """
        Sample for lmp_suffix: for a call like
            srun lmp -in lmp_production.input \
                -v has_indenter 0 -v pbc2d 0 -v mpiio 0 \
                -v thermo_frequency 1000 -v reinitialize_velocities 0 \
                -v use_colvars 0 -v productionSteps 1000 \
                -v baseName 377_SDS_on_AU_111_51x30x2_monolayer \
                -v dataFile 377_SDS_on_AU_111_51x30x2_monolayer.lammps
        set lmp_suffix='-v has_indenter 0 -v reinitialize_velocities 0 \
            -v baseName {baseName:s} -v dataFile {dataFile:s} ...'
        """
        consecutive_fw_list = []
        
        get_input_files_ft = GetFilesTask( {'identifiers':    ['lmp_header.input',
                                                               'lmp_production.input',
                                                               'lmp_production_mixed.input',
                                                               'extract_thermo.sh' , 
                                                               system_name + '_nptEquilibrated.lammps'],
                                            'new_file_names': ['lmp_header.input',
                                                               'lmp_production.input',
                                                               'lmp_production_mixed.input',
                                                               'extract_thermo.sh',
                                                               'datafile.lammps']} )
        
        get_input_files_fw = Firework( get_input_files_ft,
            spec = {
                "_category":  "nemo_noqueue",
                "system_name" : system_name,
                "_files_out":  {"lmp_header":'lmp_header.input', 
                                "lmp_production":'lmp_production.input', 
                                'lmp_production_mixed':'lmp_production_mixed.input',
                                "extract_thermo":'extract_thermo.sh',
                                "datafile": 'datafile.lammps'},
                "step"   :     "get_files",
                },
            name=(system_name + "get_files")
                                      
        )
        consecutive_fw_list.append(get_input_files_fw)
        
        lmp_cmd = ' '.join((
            'module load lammps/08May19-git-master-gnu-7.3-openmpi-3.1-colvars-08May19;',
            'mpirun ${{MPIRUN_OPTIONS}} lmp -in '+ 'lmp_production.input' , lmp_suffix_template.format(
                baseName= system_name, dataFile='datafile.lammps' ) ) )

        lmp_ft =  ScriptTask.from_str(
            lmp_cmd,
            {
                'stdout_file':  'lmp_production.out',
                'stderr_file':  'lmp_production.err',
                'use_shell':    True,
                'fizzle_bad_rc':True
            })

        extract_thermo_ft =  ScriptTask.from_str(
            'bash extract_thermo.sh {:s} {:s}'.format(
                system_name + '_production.log',
                system_name + '_production_thermo.out'),
                {
                    'use_shell':    True, 'fizzle_bad_rc': False
                } )

        fw = Firework(
            [
                lmp_ft,
                extract_thermo_ft,
                #*tail_ft_list
            ],
            spec={
                "_category":     "nemo_queue_offline",
                "system_name":    system_name,
                "_files_in":  {"lmp_header":'lmp_header.input', 
                                "lmp_production":'lmp_production.input', 
                                'lmp_production_mixed':'lmp_production_mixed.input',
                                "extract_thermo":'extract_thermo.sh',
                                "datafile": 'datafile.lammps'},
                "step"     :      "production",
                "total_steps":    total_steps,
            },
            name="{:s}_produtcion_{:d}".format(system_name, total_steps) ,
            parents=[get_input_files_fw])
        
        consecutive_fw_list.append(add_output_files_fw)
        
        parent_links = { consecutive_fw_list[i] : consecutive_fw_list[i+1] \
                        for i in range(len(consecutive_fw_list)-1) }

        return Workflow( consecutive_fw_list, parent_links,
            name="{:s}_production_wf".format(system_name) )
 

In [1]:
def run_minimize(inputs):
    minimize_wfs= ['' for x in range((len(inputs)))] 
    lmp_suffix_template=' '.join((
        '-v baseName {baseName:s} -v dataFile {dataFile:s} '))
    for i in range((len(inputs))):
        inputs_single=inputs[i]
        minimize_wfs[i]=minimize(inputs_single["system_name"],lmp_suffix_template)
    return FWAction(detours=minimize_wfs )

def run_nvtEquilibrate(inputs):
    nvtEquilibrate_wfs= ['' for x in range((len(inputs)))] 
    lmp_suffix_template=' '.join((
        '-v baseName {baseName:s} -v dataFile {dataFile:s} '))
    for i in range((len(inputs))):
        inputs_single=inputs[i]
        minimize_wfs[i]=nvtEquilibrate(inputs_single["system_name"],lmp_suffix_template)
    return FWAction(detours=nvtEquilibrate_wfs )

def run_nptEquilibrate(inputs):
    nptEquilibrate_wfs= ['' for x in range((len(inputs)))] 
    lmp_suffix_template=' '.join((
        '-v baseName {baseName:s} -v dataFile {dataFile:s} '))
    for i in range((len(inputs))):
        inputs_single=inputs[i]
        nptEquilibrate_wfs[i]=nptEquilibrate(inputs_single["system_name"],lmp_suffix_template)
    return FWAction(detours=nptEquilibrate_wfs )

def production(inputs):
    production_wfs= ['' for x in range((len(inputs)))] 
    lmp_suffix_template=' '.join((
        '-v baseName {baseName:s} -v dataFile {dataFile:s} '))
    for i in range((len(inputs))):
        inputs_single=inputs[i]
        production_wfs[i]=production(inputs_single["system_name"],lmp_suffix_template)
    return FWAction(detours=production_wfs )

In [ ]:
def minimize(system_name,
        lmp_suffix_template='-v baseName {baseName:s} -v dataFile {dataFile:s}'):
        """
        Sample for lmp_suffix: for a call like
            srun lmp -in lmp_minimization.input \
                -v has_indenter 1 -v robust_minimization 0 -v pbc2d 1 \
                -v baseName 377_SDS_on_AU_111_51x30x2_monolayer \
                -v dataFile 377_SDS_on_AU_111_51x30x2_monolayer.lammps
        set lmp_suffix='-v has_indenter 1 -v robust_minimization 0 -v pbc2d 1 \
            -v baseName {baseName:s} -v dataFile {dataFile:s}'
        """
        
        consecutive_fw_list = []

        get_input_files_ft = GetFilesTask( {
                'identifiers':    [ 'lmp_header.input', 
                                   'lmp_minimization.input',
                                   'extract_thermo.sh',
                                   system_name+'_psfgen.data' ] ,
                'new_file_names': [ 'lmp_header.input',
                                   'lmp_minimization.input',
                                   'extract_thermo.sh',
                                   'datafile.lammps']} )
        
        
        get_input_files_fw = Firework( get_input_files_ft,
            spec = {
                "_category":  "nemo_noqueue",
                "system_name" : system_name,
                "_files_out":  {"lmp_header":'lmp_header.input', 
                                "lmp_minimization":'lmp_minimization.input', 
                                "extract_thermo":'extract_thermo.sh', 
                                "datafile": 'datafile.lammps'},
                "step"   :     "get_files",
                },
            name=(system_name + "get_files") 
        )
        consecutive_fw_list.append(get_input_files_fw)
        
        
        lmp_cmd= ' '.join((
            'module load LAMMPS;',
            'mpirun ${{MPIRUN_OPTIONS}} lmp -in '+ 'lmp_minimization.input', 
                lmp_suffix_template.format(baseName= ex system_name, dataFile='datafile.lammps' ) ))

        minimization_ft =  ScriptTask.from_str(
            lmp_cmd,
            {
                'stdout_file':  'lmp_minimization.out',
                'stderr_file':  'lmp_minimization.err',
                'use_shell':    True,
                'fizzle_bad_rc':True
            })

        extract_thermo_ft =  ScriptTask.from_str(
            'bash extract_thermo.sh {:s} {:s}'.format(
                system_name + '_minimization.log',
                system_name + '_minimization_thermo.out'),
                {
                    'use_shell':    True, 'fizzle_bad_rc': False
                } )

        fw = Firework(
            [
                minimization_ft,
                extract_thermo_ft
            ],
            spec={
                "_category":    "nemo_queue_offline",
                "_files_in":  {"lmp_header":'lmp_header.input', 
                                "lmp_minimization":'lmp_minimization.input', 
                                "extract_thermo":'extract_thermo.sh', 
                                "datafile": 'datafile.lammps'},
                "_files_out":  {"lmp_minimized":system_name + '_minimized.lammps'},
                "system_name": system_name,
                "step"     : "minimization"
            },
            name="{:s}_minimzation".format(system_name),
            parents=[get_input_files_fw] )
        
        consecutive_fw_list.append(fw)
        
        add_output_files_ft= AddFilesTask({'compress':True ,
                                           'identifier': [ system_name + '_minimized.lammps'], 
                                           'paths': [ system_name + '_minimized.lammps']})
        
        
        add_output_files_fw = Firework(add_output_files_ft,
            spec = {
                "_category":  "nemo_noqueue",
                "system_name" : system_name,
                "_files_in":  { "lmp_minimized":system_name + '_minimized.lammps' },
                "step"   :     "add_output_files",
                },
            name=(system_name + "add_output_files"),
            parents=[fw]
       )
        consecutive_fw_list.append(add_output_files_fw)
        
        parent_links = { consecutive_fw_list[i] : consecutive_fw_list[i+1] \
                        for i in range(len(consecutive_fw_list)-1) }

        return Workflow( consecutive_fw_list, parent_links,
            name="{:s}_min_wf".format(system_name) )

    
def nvtEquilibrate(system_name,
        lmp_suffix_template='-v baseName {baseName:s} -v dataFile {dataFile:s}'):
        """
        Sample for lmp_suffix: for a call like
            srun lmp -in lmp_equilibration_nvt.input \
                -v has_indenter 0 -v pbc2d 0 -v reinitialize_velocities 1\
                -v baseName 377_SDS_on_AU_111_51x30x2_monolayer \
                -v dataFile 377_SDS_on_AU_111_51x30x2_monolayer.lammps
        set lmp_suffix='-v has_indenter 0 -v reinitialize_velocities 1 \
            -v baseName {baseName:s} -v dataFile {dataFile:s}'
        """

        consecutive_fw_list = []
        
        get_input_files_ft = GetFilesTask( {
                'identifiers':    ['lmp_header.input', 
                                   'lmp_equilibration_nvt.input', 
                                   'extract_thermo.sh', 
                                   system_name + '_minimized.lammps'
                                  ],
                'new_file_names': ['lmp_header.input', 
                                   'lmp_equilibration_nvt.input',
                                   'extract_thermo.sh',
                                   'datafile.lammps']} )
        
        get_input_files_fw = Firework( get_input_files_ft,
            spec = {
                "_category":  "nemo_noqueue",
                "system_name" : system_name,
                "_files_out":  {"lmp_header":'lmp_header.input', 
                                "lmp_equilibration":'lmp_equilibration.input', 
                                "extract_thermo":'extract_thermo.sh', 
                                "datafile": 'datafile.lammps'},
                "step"   :     "get_files",
                },
            name=(system_name + "get_files") 
        )
        consecutive_fw_list.append(get_input_files_fw)
  
        lmp_cmd = ' '.join((
            'module load LAMMPS;',
            'mpirun ${{MPIRUN_OPTIONS}} lmp -in '+ 'lmp_equilibration_nvt.input' , lmp_suffix_template.format(
                baseName=system_name, dataFile='datafile.lammps' ) ) )

        lmp_ft =  ScriptTask.from_str(
            lmp_cmd,
            {
                'stdout_file':  'lmp_nvtEquilibration.out',
                'stderr_file':  'lmp_nvtEquilibration.err',
                'use_shell':    True,
                'fizzle_bad_rc':True
            })

        extract_thermo_ft =  ScriptTask.from_str(
            'bash extract_thermo.sh {:s} {:s}'.format(
                system_name + '_nvtEquilibration.log',
                system_name + '_nvtEquilibration_thermo.out'),
                {
                    'use_shell':    True, 'fizzle_bad_rc': False
                } )

        fw = Firework(
            [
                lmp_ft,
                extract_thermo_ft
            ],
            spec={
             
                "_category":    "nemo_queue_offline",
                "system_name": system_name,
                "_files_in":  { "lmp_header":'lmp_header.input', 
                                "lmp_equilibration":'lmp_equilibration_nvt.input', 
                                "extract_thermo":'extract_thermo.sh', 
                                "datafile": 'datafile.lammps'},
                "_files_out":  {"lmp_nvt": system_name + '_nvtEquilibrated.lammps'},
                "step"     :   "equilibration_nvt",
            },
            name="{:s}_equilibration_nvt".format(system_name),
            parents=[get_input_files_fw] )
        
        
         consecutive_fw_list.append(fw)
    
         add_output_files_ft= AddFilesTask({'compress':True ,
                                           'identifier': [ system_name + '_nvtEquilibrated.lammps'],
                                           'paths': [ system_name + '_nvtEquilibrated.lammps']})
        
        
        add_output_files_fw = Firework(add_output_files_ft,
            spec = {
                "_category":  "nemo_noqueue",
                "system_name" : system_name,
                "_files_in":  { "lmp_nvt": system_name + '_nvtEquilibrated.lammps'},
                "step"   :     "add_output_files",
                },
            name=(system_name + "add_output_files"),
            parents=[fw]
       )
        consecutive_fw_list.append(add_output_files_fw)
        
        parent_links = { consecutive_fw_list[i] : consecutive_fw_list[i+1] \
                        for i in range(len(consecutive_fw_list)-1) }

        return Workflow( consecutive_fw_list, parent_links,
            name="{:s}_nvt_wf".format(system_name) )
    


def nptEquilibrate(system_name,
        lmp_suffix_template='-v baseName {baseName:s} -v dataFile {dataFile:s}'):
        """
        Sample for lmp_suffix: for a call like
            srun lmp -in lmp_equilibration_npt.input \
                -v has_indenter 1 -v pbc2d 0 -v reinitialize_velocities 1\
                -v baseName 377_SDS_on_AU_111_51x30x2_monolayer \
                -v dataFile 377_SDS_on_AU_111_51x30x2_monolayer.lammps
        set lmp_suffix='-v has_indenter 1 -v reinitialize_velocities 1 \
            -v baseName {baseName:s} -v dataFile {dataFile:s}'
        """
        consecutive_fw_list = []

        get_input_files_ft = GetFilesTask( {
                'identifiers':    ['lmp_header.input', 
                                   'lmp_equilibration_npt.input', 
                                   'extract_thermo.sh',
                                   system_name + '_nvtEquilibrated.lammps'
                                  ],
                'new_file_names': [ 'lmp_header.input', 
                                   'lmp_equilibration_npt.input', 
                                   'extract_thermo.sh',
                                   'datafile.lammps']} )
        
        get_input_files_fw = Firework( get_input_files_ft,
            spec = {
                "_category":  "nemo_noqueue",
                "system_name" : system_name,
                "_files_out":  {"lmp_header":'lmp_header.input', 
                                "lmp_equilibration":'lmp_equilibration_npt.input', 
                                "extract_thermo":'extract_thermo.sh',
                                "datafile": 'datafile.lammps'},
                "step"   :     "get_files",
                },
            name=(system_name + "get_files")
                                      
        )
        consecutive_fw_list.append(get_input_files_fw)
        
        lmp_cmd = ' '.join((
            'module load LAMMPS;',
            'mpirun ${{MPIRUN_OPTIONS}} lmp -in '+ 'lmp_equilibration_npt.input' , lmp_suffix_template.format(
                baseName=system_name, dataFile='datafile.lammps' ) ) )

        lmp_ft =  ScriptTask.from_str(
            lmp_cmd,
            {
                'stdout_file':  'lmp_nptEquilibration.out',
                'stderr_file':  'lmp_nptEquilibration.err',
                'use_shell':    True,
                'fizzle_bad_rc':True
            })

        extract_thermo_ft =  ScriptTask.from_str(
            'bash extract_thermo.sh {:s} {:s}'.format(
                system_name + '_nptEquilibration.log',
                system_name + '_nptEquilibration_thermo.out'),
                {
                    'use_shell':    True, 'fizzle_bad_rc': False
                } )

        fw = Firework(
            [
                lmp_ft,
                extract_thermo_ft
            ],
            spec={
                "_category":   "nemo_queue_offline",
                "system_name": system_name,
                "step"     :   "equilibration_npt",
                "_files_in":  {"lmp_header":'lmp_header.input', 
                                "lmp_equilibration":'lmp_equilibration_npt.input', 
                                "extract_thermo":'extract_thermo.sh',
                                "datafile": 'datafile.lammps'},
                "_files_out": {"lmp_npt":  system_name + '_nptEquilibrated.lammps'}
            },
            name="{:s}_equilibration_npt".format(system_name),
            parents=[get_input_files_fw])
        
         consecutive_fw_list.append(fw)
        
        add_output_files_ft= AddFilesTask({'compress':True ,
                                           'identifier': [ system_name + '_nptEquilibrated.lammps'], 
                                           'paths': [ system_name + '_nptEquilibrated.lammps']})
        
        add_output_files_fw = Firework(add_output_files_ft,
            spec = {
                "_category":  "nemo_noqueue",
                "system_name" : system_name,
                "_files_in":  { "lmp_npt": system_name + '_nptEquilibrated.lammps'},
                "step"   :     "add_output_files",
                },
            name=(system_name + "add_output_files"),
            parents=[fw]
       )
        consecutive_fw_list.append(add_output_files_fw)
        
        parent_links = { consecutive_fw_list[i] : consecutive_fw_list[i+1] \
                        for i in range(len(consecutive_fw_list)-1) }

        return Workflow( consecutive_fw_list, parent_links,
            name="{:s}_npt_wf".format(system_name) )

def production(system_name,
        total_steps      = 5000000, # time steps, 5 mio ~ 10 ns
        lmp_suffix_template=' '.join((
            '-v baseName {baseName:s} -v dataFile {dataFile:s}',
            '-v has_indenter 0 -v pbc2d 0 -v mpiio 0 -v use_colvars 0')) ):
        """
        Sample for lmp_suffix: for a call like
            srun lmp -in lmp_production.input \
                -v has_indenter 0 -v pbc2d 0 -v mpiio 0 \
                -v thermo_frequency 1000 -v reinitialize_velocities 0 \
                -v use_colvars 0 -v productionSteps 1000 \
                -v baseName 377_SDS_on_AU_111_51x30x2_monolayer \
                -v dataFile 377_SDS_on_AU_111_51x30x2_monolayer.lammps
        set lmp_suffix='-v has_indenter 0 -v reinitialize_velocities 0 \
            -v baseName {baseName:s} -v dataFile {dataFile:s} ...'
        """
        consecutive_fw_list = []
        
        get_input_files_ft = GetFilesTask( {'identifiers':    ['lmp_header.input',
                                                               'lmp_production.input',
                                                               'lmp_production_mixed.input',
                                                               'extract_thermo.sh' , 
                                                               system_name + '_nptEquilibrated.lammps'],
                                            'new_file_names': ['lmp_header.input',
                                                               'lmp_production.input',
                                                               'lmp_production_mixed.input',
                                                               'extract_thermo.sh',
                                                               'datafile.lammps']} )
        
        get_input_files_fw = Firework( get_input_files_ft,
            spec = {
                "_category":  "nemo_noqueue",
                "system_name" : system_name,
                "_files_out":  {"lmp_header":'lmp_header.input', 
                                "lmp_production":'lmp_production.input', 
                                'lmp_production_mixed':'lmp_production_mixed.input',
                                "extract_thermo":'extract_thermo.sh',
                                "datafile": 'datafile.lammps'},
                "step"   :     "get_files",
                },
            name=(system_name + "get_files")
                                      
        )
        consecutive_fw_list.append(get_input_files_fw)
        
        
       
        lmp_cmd = ' '.join((template_lmp_cmd.format(inputFile = 'lmp_production.input'), 
                lmp_suffix_template.format(baseName=system_name, dataFile='datafile.lammps' )))

        lmp_ft =  ScriptTask.from_str(
            lmp_cmd,
            {
                'stdout_file':  'lmp_production.out',
                'stderr_file':  'lmp_production.err',
                'use_shell':    True,
                'fizzle_bad_rc':True
            })

        extract_thermo_ft =  ScriptTask.from_str(
            'bash extract_thermo.sh {:s} {:s}'.format(
                system_name + '_production.log',
                system_name + '_production_thermo.out'),
                {
                    'use_shell':    True, 'fizzle_bad_rc': False
                } )

        fw = Firework(
            [
                lmp_ft,
                extract_thermo_ft,
                #*tail_ft_list
            ],
            spec={
                "_category":     "nemo_queue_offline",
                "system_name":    system_name,
                "_files_in":  {"lmp_header":'lmp_header.input', 
                                "lmp_production":'lmp_production.input', 
                                'lmp_production_mixed':'lmp_production_mixed.input',
                                "extract_thermo":'extract_thermo.sh',
                                "datafile": 'datafile.lammps'},
                "step"     :      "production",
                "total_steps":    total_steps,
            },
            name="{:s}_produtcion_{:d}".format(system_name, total_steps) ,
            parents=[get_input_files_fw])
        
                consecutive_fw_list.append(add_output_files_fw)
        
        parent_links = { consecutive_fw_list[i] : consecutive_fw_list[i+1] \
                        for i in range(len(consecutive_fw_list)-1) }

        return Workflow( consecutive_fw_list, parent_links,
            name="{:s}_production_wf".format(system_name) )
        